In [1]:
import pyreadstat
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, OneHotEncoder, MinMaxScaler, StandardScaler
import numpy as np

# Import Data

In [15]:
# 국건영 데이터 (학습 데이터) 불러오기
diet_raw_data, diet_meta = pyreadstat.read_sav('HN22/hn22_24rc_240111/hn22_24rc_240111.sav')
diet_meta_df = pd.DataFrame([diet_meta.column_names, diet_meta.column_labels]).transpose()

user_raw_data, user_meta = pyreadstat.read_sav('HN22/hn22_ALL/hn22_ALL.sav')
user_meta_df = pd.DataFrame([user_meta.column_names, user_meta.column_labels]).transpose()

HN_df = pd.read_excel('../data/HN22/HN22.xlsx')[['ID','sex','age','HE_ht','HE_wt','PA']]

# 매핑 데이터 불러오기
Fcode_mapping = pd.read_excel('HN22/code_mapping.xlsx', sheet_name='FCODE')
Fcode_mapping['N_FCODE'] = Fcode_mapping['N_FCODE'].astype(int)

# 레시피 데이터 불러오기
KRecipe_N = pd.read_excel('Recipe/KRecipeDB_N.xlsx', sheet_name='Nutrient')
KRecipe_N.columns = ['N_DCODE','NF_EN','NF_CHO','NF_PROT','NF_FAT','NF_SUGAR','NF_TDF','NF_NA','NF_CHOL','NF_SFA']

KRecipe_I = pd.read_excel('Recipe/KRecipeDB_N.xlsx', sheet_name='Ingredients')
KRecipe_C = pd.read_excel('Recipe/KRecipeDB_N.xlsx', sheet_name='Category')
KRecipe_R = pd.read_excel('Recipe/KRecipeDB_N.xlsx', sheet_name='Recipe')

In [16]:
user_raw_data

,mod_d,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,...,N_RETIN,N_B1,N_B2,N_NIAC,N_FOLATE,N_VITC,LF_secur_y,LF_BUYER,LF_SAFE,N_DUSUAL
0,2024.01.11.,YA01220302,YA012203,2022.0,1.0,1.0,2.0,YA01,2.0,56.0,...,383.340000,0.814587,1.182300,10.042185,291.264557,32.441192,1.0,NaN,1.0,3
1,2024.01.11.,YA01220303,YA012203,2022.0,1.0,1.0,2.0,YA01,1.0,30.0,...,732.385000,1.829417,2.017004,18.342991,329.385371,79.903529,1.0,NaN,1.0,3
2,2024.01.11.,YA01220304,YA012203,2022.0,1.0,1.0,2.0,YA01,2.0,25.0,...,0.000000,0.262020,1.379288,3.042505,69.694336,2.776080,1.0,1.0,1.0,2
3,2024.01.11.,YA01236501,YA012365,2022.0,1.0,1.0,2.0,YA01,1.0,66.0,...,100.898983,1.338621,2.325439,18.325182,633.687278,112.775603,1.0,NaN,1.0,2
4,2024.01.11.,YA01236502,YA012365,2022.0,1.0,1.0,2.0,YA01,2.0,62.0,...,128.641794,1.317710,2.564474,17.852091,690.090821,121.924030,1.0,1.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6260,2024.01.11.,YR38363902,YR383639,2022.0,8.0,1.0,2.0,YR38,2.0,26.0,...,68.880000,0.325227,1.696439,4.784589,173.415000,40.281930,1.0,2.0,2.0,2
6261,2024.01.11.,YR38365901,YR383659,2022.0,8.0,1.0,2.0,YR38,1.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.
6262,2024.01.11.,YR38368001,YR383680,2022.0,8.0,1.0,2.0,YR38,2.0,58.0,...,72.616397,1.596080,2.263162,17.249301,840.642746,320.516175,1.0,1.0,2.0,1
6263,2024.01.11.,YR38368002,YR383680,2022.0,8.0,1.0,2.0,YR38,1.0,62.0,...,20.095810,1.844611,1.904079,13.545765,541.703206,127.114502,1.0,NaN,2.0,2


# Preprocess Data

In [3]:
# 원하는 데이터만 추출
imp_column = ['ID', 'sex', 'age', 'incm', 'edu', 'occp','N_DCODE', 'N_DNAME', 'N_MEAL', 'N_FCODE', 'N_FNAME', 'N_TD_VOL', 'N_TD_WT', 'N_CD_VOL', 'N_CD_WT', 'N_FM_WT']
nutri_column = ['NF_EN','NF_CHO','NF_PROT','NF_FAT','NF_SFA','NF_CHOL','NF_TDF','NF_SUGAR','NF_NA']

diet_data = diet_raw_data[imp_column+nutri_column].copy()

def normalize_dataframe(df, columns_to_normalize, max_int_value=100):
    scaler = MinMaxScaler()
    df[columns_to_normalize] = np.log1p(df[columns_to_normalize])
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
    df[columns_to_normalize] = np.ceil(df[columns_to_normalize] * max_int_value).astype(int)
    return df

def preprocess_categorical_features(df):
    # 범주형 변수 원핫인코딩
    categorical_cols = ['sex', 'incm', 'edu', 'occp', 'PA']
    
    # 원본 데이터 보존을 위한 복사
    processed_df = df.copy()
    
    # 각 범주형 변수에 대해 원핫인코딩
    for col in categorical_cols:
        # 빈도가 낮은 범주 처리
        value_counts = processed_df[col].value_counts(normalize=True)
        rare_categories = value_counts[value_counts < 0.01].index
        processed_df[col] = processed_df[col].replace(rare_categories, 'Other')
        
        # 원핫인코딩
        encoded = pd.get_dummies(processed_df[col], prefix=col)
        processed_df = pd.concat([processed_df, encoded], axis=1)
        processed_df.drop(col, axis=1, inplace=True)
    
    return processed_df

In [ ]:
# 부피, 중량 합치기
diet_data['N_TD'] = np.where(diet_data['N_TD_VOL'].notna(), diet_data['N_TD_VOL'], diet_data['N_TD_WT'])
diet_data['N_CD'] = np.where(diet_data['N_CD_VOL'].notna(), diet_data['N_CD_VOL'], diet_data['N_CD_WT'])
diet_data = diet_data.drop(['N_TD_VOL','N_TD_WT'], axis=1)  
diet_data = diet_data.drop(['N_CD_VOL','N_CD_WT'], axis=1)
diet_data['N_CD'] = np.where(diet_data['N_CD'].notna(), diet_data['N_CD'], diet_data['N_TD'])
diet_data['N_TD'] = np.where(diet_data['N_TD'].notna(), diet_data['N_TD'], diet_data['N_CD'])

# 식품 코드 통일
diet_data['N_FCODE'] = diet_data['N_FCODE'].astype(int)
diet_data = pd.merge(diet_data, Fcode_mapping, on='N_FCODE', how='left').drop('N_FCODE',axis=1).rename(columns={'M_FCODE':'N_FCODE'})

# 음식 코드 통일
diet_data = diet_data.rename(columns={'N_DCODE':'O_DCODE'})
name_to_code = {name: f"{i:02d}" for i, name in enumerate(diet_data['N_DNAME'].unique())}
diet_data['N_DCODE'] = diet_data['O_DCODE'].astype(str) + diet_data['N_DNAME'].map(name_to_code)
diet_data = diet_data.drop(columns=['O_DCODE']).drop_duplicates().reset_index(drop=True)
diet_data = diet_data.fillna(0)

# 간식 제외
diet_data['second_third_digits'] = diet_data['N_DCODE'].str[1:3]
exclude_list = ['19', '49', '20', '50', '21', '51', '22', '52', '23', '53', '24', '54']
diet_data = diet_data[~diet_data['second_third_digits'].isin(exclude_list)].drop(columns=['second_third_digits'])

# Define Data 

## Node

In [5]:
# User Node
user_data = pd.merge(diet_data[['ID', 'incm', 'edu', 'occp']].drop_duplicates(), HN_df, on='ID', how='left').dropna()
user_data[['sex','age','HE_ht','HE_wt','PA']] = user_data[['sex','age','HE_ht','HE_wt','PA']].astype(float)

user_columns_to_normalize = ['age', 'HE_ht', 'HE_wt']
user_data = normalize_dataframe(user_data, user_columns_to_normalize)

# 범주형 변수 전처리
user_data = preprocess_categorical_features(user_data)

user_data.head(5)

,ID,age,HE_ht,HE_wt,sex_1.0,sex_2.0,incm_1.0,incm_2.0,incm_3.0,incm_4.0,...,occp_2.0,occp_3.0,occp_4.0,occp_5.0,occp_6.0,occp_7.0,PA_1.0,PA_2.0,PA_3.0,PA_4.0
0,YA01220302,91,76,70,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
1,YA01220303,75,90,81,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
2,YA01220304,70,86,69,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
3,YA01236501,95,87,74,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
4,YA01236502,94,80,66,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0


In [6]:
# Food
food_data = diet_data[['N_DCODE','N_TD','N_CD']+nutri_column].drop_duplicates().reset_index(drop=True)
for nutri in nutri_column:
    food_data[nutri] = food_data['N_CD'] * food_data[nutri] / food_data['N_TD']
food_data = food_data.drop(['N_TD','N_CD'], axis=1)
food_data = food_data.groupby('N_DCODE').sum().reset_index()
food_data = pd.concat([food_data, KRecipe_N], axis=0)
food_data['N_DCODE'] = food_data['N_DCODE'].astype(str)
food_data = normalize_dataframe(food_data, ['NF_EN', 'NF_CHO', 'NF_PROT', 'NF_FAT', 'NF_SFA', 'NF_CHOL', 'NF_TDF', 'NF_SUGAR', 'NF_NA'])

food_data.head(5)

,N_DCODE,NF_EN,NF_CHO,NF_PROT,NF_FAT,NF_SFA,NF_CHOL,NF_TDF,NF_SUGAR,NF_NA
0,10101627,54,49,37,23,15,0,32,18,22
1,101017627,57,53,41,27,20,0,38,22,26
2,101027633,57,53,40,28,20,0,39,46,31
3,101028716,58,53,40,28,20,0,38,47,31
4,10104211,47,40,32,31,20,56,23,18,50


In [7]:
# Ingredient
ingre_data = diet_data[['N_FCODE']].drop_duplicates()
ingre_data = pd.concat([ingre_data, KRecipe_I[['N_FCODE']]], axis=0)

def get_feature(code):
    if code[0] == 'R':
        return code[1] + code[2:4]
    elif code[0] == 'P':
        return '4' + code[2:4]
    else:
        return '5' + code[2:4]

ingre_data['feature'] = ingre_data['N_FCODE'].apply(get_feature)
ingre_data['ingredient_type'] = ingre_data['feature'].apply(
    lambda x: 'flavor' if x[0] in ['1', '2', '3'] and x[1:3] == '18' else 'main')

encoded_features = pd.get_dummies(ingre_data['feature'], prefix='feature')
ingre_data = pd.concat([ingre_data, encoded_features], axis=1).drop('feature', axis=1)

ingre_data_main = ingre_data[ingre_data['ingredient_type'] == 'main'].drop('ingredient_type', axis=1)
ingre_data_main = ingre_data_main.loc[:, (ingre_data_main.sum(axis=0) != 0)]

ingre_data_flavor = ingre_data[ingre_data['ingredient_type'] == 'flavor'].drop('ingredient_type', axis=1)
ingre_data_flavor = ingre_data_flavor.loc[:, (ingre_data_flavor.sum(axis=0) != 0)]

ingre_data_main.head(5)

,N_FCODE,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_112,...,feature_510,feature_511,feature_512,feature_513,feature_514,feature_515,feature_516,feature_517,feature_520,feature_527
0,R106-208007947-0000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,R114-020000000-0000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,R106-191040001-0000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,R106-053002201-0000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,P104-601060100-0002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Meal
meal_data = diet_data[['N_MEAL']].drop_duplicates()
meal_data.head(5)

,N_MEAL
0,2.0
15,3.0
41,4.0
77,1.0


## Edge

In [9]:
# user - food
intake_data = diet_data[['ID', 'N_DCODE','N_TD','N_CD']]
intake_data['edge_weight'] = intake_data['N_TD']/intake_data['N_CD']*100
intake_data = intake_data.drop(['N_TD','N_CD'], axis=1).drop_duplicates().reset_index(drop=True)
intake_data = intake_data.groupby(['ID','N_DCODE']).mean().reset_index()
intake_data['N_DCODE'] = intake_data['N_DCODE'].astype(str)
intake_data = normalize_dataframe(intake_data, ['edge_weight'])
intake_data = intake_data[intake_data['edge_weight'] > 60].copy().drop_duplicates()
intake_data['edge_weight'] = 1

real_user = user_data['ID'].tolist()
filtered_intake_data = intake_data[intake_data['ID'].isin(real_user)].copy()

filtered_intake_data.head(5)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28096\1008640610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intake_data['edge_weight'] = intake_data['N_TD']/intake_data['N_CD']*100


,ID,N_DCODE,edge_weight
2,YA01220302,5025709,1
3,YA01220302,5151306,1
4,YA01220302,5152808,1
5,YA01220302,7118101,1
9,YA01220303,5150215,1


In [10]:
# food - food
def create_food_pairs(meal_data):
    pairs = []
    for _, group in meal_data.groupby(['ID', 'N_MEAL']):
        foods = group['N_DCODE'].tolist()
        pairs.extend([(food1, food2) for i, food1 in enumerate(foods) for food2 in foods[i+1:]])
    pairs_df = pd.DataFrame(pairs, columns=['food1', 'food2'])
    pair_counts = pairs_df.groupby(['food1', 'food2']).size().reset_index(name='edge_weight')
    return pair_counts

meal_data = diet_data[['ID', 'N_DCODE', 'N_MEAL']].drop_duplicates().reset_index(drop=True)
food_pairs = create_food_pairs(meal_data)
food_pairs['food1'] = food_pairs['food1'].astype(str)
food_pairs['food2'] = food_pairs['food2'].astype(str)

food_pairs['edge_weight'] = 1

food_pairs.head(5)

,food1,food2,edge_weight
0,10101627,110221318,1
1,10101627,13533143,1
2,10101627,5151306,1
3,101017627,5081055,1
4,101017627,50915176,1


In [11]:
# food - ingredient
food_ing_data = diet_data[['N_DCODE', 'N_FCODE', 'N_FM_WT']].drop_duplicates().reset_index(drop=True)
food_ing_data = food_ing_data.groupby(['N_DCODE','N_FCODE']).mean().reset_index()
KRecipe_R = KRecipe_R.rename(columns={'edge_weight':'N_FM_WT'})
food_ing_data = pd.concat([food_ing_data, KRecipe_R], axis=0)
food_ing_data['N_DCODE'] = food_ing_data['N_DCODE'].astype(str)
food_ing_data['N_FCODE'] = food_ing_data['N_FCODE'].astype(str)

food_ing_g = food_ing_data.groupby('N_DCODE')['N_FM_WT'].sum().reset_index()
food_ing_g = food_ing_g.rename(columns={'N_FM_WT': 'total_weight'})
food_ing_data_r = pd.merge(food_ing_data, food_ing_g, on='N_DCODE')
food_ing_data_r['edge_weight'] = food_ing_data_r['N_FM_WT'] / food_ing_data_r['total_weight'] * 100
food_ing_data_r = food_ing_data_r.dropna()

def ingredient_type(code):
    feature = get_feature(code)
    return 'flavor' if feature[1:3] == '18' else 'main'
food_ing_data_r['ingredient_type'] = food_ing_data_r['N_FCODE'].apply(ingredient_type)

food_ing_main = food_ing_data_r[(food_ing_data_r['ingredient_type'] == 'main') & (food_ing_data_r['edge_weight'] >= 50)].copy()
food_ing_main['edge_weight'] = 1
food_ing_main = food_ing_main.drop(['N_FM_WT', 'total_weight', 'ingredient_type'], axis=1)

food_ing_flavor = food_ing_data_r[food_ing_data_r['ingredient_type'] == 'flavor'].copy()
food_ing_flavor['edge_weight'] = 1
food_ing_flavor = food_ing_flavor.drop(['N_FM_WT', 'total_weight', 'ingredient_type'], axis=1)

food_ing_flavor.head(5)

,N_DCODE,N_FCODE,edge_weight
32,101042520,R318-020000000-0000,1
38,101044255,R318-020000000-0000,1
45,101067330,R318-020000000-0000,1
51,101091632,R318-020000000-0000,1
80,10110211,R318-020000000-0000,1


In [12]:
# food - time
food_time_data = diet_data.groupby(['N_DCODE', 'N_MEAL']).size().reset_index(name='count')
food_time_data['edge_weight'] = food_time_data.groupby('N_DCODE')['count'].transform(lambda x: x / x.sum())*100
food_time_data = food_time_data.drop('count', axis=1)
food_time_data['N_DCODE'] = food_time_data['N_DCODE'].astype(str)
food_time_data = food_time_data[food_time_data['edge_weight'] > 30].copy().drop_duplicates()
food_time_data['edge_weight'] = 1

food_time_data.head(5)

,N_DCODE,N_MEAL,edge_weight
0,10101627,1.0,1
1,101017627,1.0,1
2,101017627,3.0,1
3,101027633,1.0,1
4,101027633,2.0,1


# Save Data

In [13]:
dfs = {
    'diet_data': diet_data,
    'user_data': user_data,
    'food_data': food_data,
    'ingre_data_main': ingre_data_main,
    'ingre_data_flavor': ingre_data_flavor,
    'meal_data': meal_data,
    'intake_data': filtered_intake_data,
    'food_pairs': food_pairs,
    'food_ing_main': food_ing_main,
    'food_ing_flavor': food_ing_flavor,
    'food_time_data': food_time_data
}

with pd.ExcelWriter('diet_data.xlsx') as writer:
    for sheet_name, df in dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)